# Taller inicial de Julia

## Environments and the Package manger (Pkg)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Parameters")
Pkg.add("Unitful")
Pkg.add("UnitfulAstro")
Pkg.add("Zygote")
Pkg.add("DataFrames")
Pkg.add("DifferentialEquations")
# Pkg.develop("GalacticDynamics")

## Arrays

In [33]:
# Arrays
a = [1.0, 2.0, 3.0]
@show a typeof(a)
@show typeof(a)<:Vector{Integer}
@show typeof(a)<:Vector{Float64};


a = [1.0, 2.0, 3.0]
typeof(a) = Vector{Float64}
typeof(a) <: Vector{Integer} = false
typeof(a) <: Vector{Float64} = true


In [35]:
 a[end]==a[3]

true

In [36]:
@show sum(a)
b = 2a
@show c = a + b
@show c = a + b.*b
@show c = a .+ b'b;

sum(a) = 6.0
c = a + b = [3.0, 6.0, 9.0]
c = a + b .* b = [5.0, 18.0, 39.0]
c = a .+ b' * b = [57.0, 58.0, 59.0]


In [40]:
a'a

14.0

In [41]:
m = [a 2a 3a]
display(m)
display(m')

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  4.0  6.0
 3.0  6.0  9.0

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  2.0  3.0
 2.0  4.0  6.0
 3.0  6.0  9.0

## Structs

In [47]:
struct Evento{T<:Real,D<:Real}
    t::T
    x::D
    v::D
end

In [49]:
methods(Evento)

# 3 methods for type constructor:
 [1] Evento(; t, x, v)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] Evento(x::T, y::T) where T<:Real
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [3] Evento(t::T, x::D, v::D) where {T<:Real, D<:Real}
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:2

In [45]:
a = Evento(7, 10., 1.9)

Evento{Int64, Float64}(7, 10.0, 1.9)

In [48]:
a = Evento(0, 10.) # show what happens

MethodError: MethodError: no method matching Evento(::Int64, ::Float64)

Closest candidates are:
  Evento(::T, ::D, !Matched::D) where {T<:Real, D<:Real}
   @ Main /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:2
  Evento(::T, !Matched::T) where T<:Real
   @ Main /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1


In [50]:
Evento(x::T,y::T) where {T<:Real} = Evento(0., x, y)

Evento

In [51]:
Evento(10., 1.)

Evento{Float64, Float64}(0.0, 10.0, 1.0)

In [52]:
methods(Evento)

# 3 methods for type constructor:
 [1] Evento(; t, x, v)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] Evento(x::T, y::T) where T<:Real
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [3] Evento(t::T, x::D, v::D) where {T<:Real, D<:Real}
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:2

In [53]:
Evento(; t=9, x=7, v) = Evento(t, x, v)

Evento

In [54]:
a = Evento(t=0, x=1, v=1)

Evento{Int64, Int64}(0, 1, 1)

In [56]:
b = Evento(t=98)

UndefKeywordError: UndefKeywordError: keyword argument `v` not assigned

In [55]:
b = Evento(v=54)

Evento{Int64, Int64}(9, 7, 54)

## Parameters package

In [57]:
using Parameters
@with_kw struct Event{T<:Real,D<:Real}
    t::T = 0.0
    x::D = 5.0
    v::D
end

Event

In [58]:
methods(Event)

# 5 methods for type constructor:
 [1] Event(; t, x, v)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:545
 [2] Event(pp::Event; kws...)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:569
 [3] Event(pp::Event, di::AbstractDict)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:572
 [4] Event(pp::Event, di::Tuple{Symbol, Any}...)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:573
 [5] Event(t::T, x::D, v::D) where {T<:Real, D<:Real}
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:526

In [60]:
b = Event(v=3.0) # then Event(v=3.0)

Event{Float64, Float64}
  t: Float64 0.0
  x: Float64 5.0
  v: Float64 3.0


## Multiple Dispatch

In [61]:
ρ(x,y,z) = (x+y)*exp(π*z)

ρ (generic function with 1 method)

In [62]:
methods(ρ)

# 1 method for generic function "ρ" from Main:
 [1] ρ(x, y, z)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1

In [63]:
ρ(x::Int,y::Float64,z::T) where {T<:Real} = -10.0

ρ (generic function with 2 methods)

In [64]:
methods(ρ)

# 2 methods for generic function "ρ" from Main:
 [1] ρ(x::Int64, y::Float64, z::T) where T<:Real
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] ρ(x, y, z)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1

In [67]:
ρ(3.0,3.0,2.0)

3212.9499331485877

In [68]:
ρ(x::T, y::T, z::T) where {T<:Real} = 7

ρ (generic function with 3 methods)

In [69]:
methods(ρ)

# 3 methods for generic function "ρ" from Main:
 [1] ρ(x::Int64, y::Float64, z::T) where T<:Real
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] ρ(x::T, y::T, z::T) where T<:Real
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [3] ρ(x, y, z)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1

In [73]:
ρ(1,1,2)

7

In [74]:
x = [1, 1., 3.]

3-element Vector{Float64}:
 1.0
 1.0
 3.0

In [75]:
ρ(x...)

7

In [76]:
ρ(x::Vector{<:Real}) = ρ(x...)+10

ρ (generic function with 4 methods)

In [ ]:
methods(ρ)

In [77]:
ρ(x)

17

## Unitful Quantities

In [79]:
using Unitful, UnitfulAstro
x = rand(3)u"kpc"
v = rand(3)u"km/s"
t = 10^6*u"s"
@show x v t
u"kpc/Gyr"

x = Quantity{Float64, 𝐋, Unitful.FreeUnits{(kpc,), 𝐋, nothing}}[0.1723385083211918 kpc, 0.18044218017205882 kpc, 0.17142360472624507 kpc]
v = Quantity{Float64, 𝐋 𝐓^-1, Unitful.FreeUnits{(km, s^-1), 𝐋 𝐓^-1, nothing}}[0.3992838179610827 km s^-1, 0.5428339149885543 km s^-1, 0.3710698873002307 km s^-1]
t = 1000000 s


kpc Gyr^-1

In [80]:
a = v/t

3-element Vector{Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-2), 𝐋 𝐓^-2, nothing}}}:
  3.992838179610827e-7 km s^-2
  5.428339149885543e-7 km s^-2
 3.7106988730023074e-7 km s^-2

In [81]:
uconvert.(u"km/s/Myr",a)

3-element Vector{Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}}:
 1.2600439013688663e7 km s^-1 Myr^-1
   1.71305355556428e7 km s^-1 Myr^-1
 1.1710075075465761e7 km s^-1 Myr^-1

In [82]:
u"G"

6.6743e-11 m^3 kg^-1 s^-2

In [83]:
G = uconvert(u"kpc*Msun^-1*(km/s)^2",u"G")

4.30091727003628e-6 km^2 kpc s^-2 M⊙^-1

## Automatic Differentiation

In [84]:
using Zygote
f(x,y) = exp(x/y)*cos(3x+2y)

f (generic function with 1 method)

In [85]:
∇f(x,y) = gradient((x,y)->f(x,y), x,y)

∇f (generic function with 1 method)

In [86]:
∇f(1.,2.)

(-2.6280759982354986, -2.4771192304363403)

In [87]:
∇f(x) = gradient(x->f(x...), x)

∇f (generic function with 2 methods)

In [88]:
methods(∇f)

# 2 methods for generic function "∇f" from Main:
 [1] ∇f(x)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] ∇f(x, y)
     @ /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:1

In [89]:
@show ∇f([1.,2.])[1]  ∇f(1.,2.)

(∇f([1.0, 2.0]))[1] = [-2.6280759982354986, -2.4771192304363403]
∇f(1.0, 2.0) = (-2.6280759982354986, -2.4771192304363403)


(-2.6280759982354986, -2.4771192304363403)

In [90]:
f(x,y) = exp(x)*exp(7y)
Df(x,y) = [1,7]*f(x,y)
∇f(x,y) = gradient((x,y)->f(x,y), x,y)
df(x,y) = gradient(f, x,y)

df (generic function with 1 method)

In [91]:
x = [5.,-7.]
@show Df(x...) ∇f(x...) df(x...)
@show Df(x...) .== ∇f(x...);
@show Df(x...) .== df(x...);

Df(x...) = [7.781132241133797e-20, 5.446792568793657e-19]
∇f(x...) = 

(7.781132241133797e-20, 5.446792568793657e-19)
df(x...) = (7.781132241133797e-20, 5.446792568793657e-19)
Df(x...) .== ∇f(x...) = 

Bool[1, 1]
Df(x...) .== df(x...) = Bool[1, 1]


## Data Frames

In [92]:
using DataFrames
n=20
data = DataFrame(id=collect(1:n), x=rand(n), y=rand(n), z=rand(n))

Row,id,x,y,z
,Int64,Float64,Float64,Float64
1,1,0.489944,0.0417274,0.493067
2,2,0.18897,0.487114,0.465865
3,3,0.323223,0.828824,0.258505
4,4,0.457105,0.630828,0.748485
5,5,0.828613,0.301434,0.753773
6,6,0.329394,0.0569095,0.791413
7,7,0.837742,0.319436,0.00364748
8,8,0.131976,0.693703,0.734074
9,9,0.263305,0.418606,0.30855


In [93]:
deleteat!(data, [3,5,7])

Row,id,x,y,z
,Int64,Float64,Float64,Float64
1,1,0.489944,0.0417274,0.493067
2,2,0.18897,0.487114,0.465865
3,4,0.457105,0.630828,0.748485
4,6,0.329394,0.0569095,0.791413
5,8,0.131976,0.693703,0.734074
6,9,0.263305,0.418606,0.30855
7,10,0.482631,0.683583,0.0822296
8,11,0.916244,0.184676,0.154271
9,12,0.0146526,0.94227,0.800638


## Galactic Dynamics 

In [94]:
using GalacticDynamics

In [96]:
pot = Plummer(34u"Msun", 10.0*u"pc")
pot₂ = NFW(10^3*u"Msun", 5.0*u"kpc")
@show pot pot₂ pot+pot₂

pot = Plummer{Int64, Float64}
  m: Int64 34
  a: Float64 0.01

pot₂ = NFW{Int64, Float64}
  m: Int64 1000
  a: Float64 5.0



pot + pot₂ = GalacticDynamics.AbstractPotential[Plummer{Int64, Float64}
  m: Int64 34
  a: Float64 0.01
, NFW{Int64, Float64}
  m: Int64 1000
  a: Float64 5.0
]


2-element Vector{GalacticDynamics.AbstractPotential}:
 Plummer{Int64, Float64}
  m: Int64 34
  a: Float64 0.01

 NFW{Int64, Float64}
  m: Int64 1000
  a: Float64 5.0


In [97]:
x=[5.0, 1.0, -2.0]u"pc"
@show potential(pot,x)
@show acceleration(pot,x)
@show potential(pot₂,x)
@show acceleration(pot₂,x)
@show potential(pot+pot₂,x)
@show acceleration(pot+pot₂,x);

potential(pot, x) = -0.01282532353900881 km^2 s^-2
acceleration(pot, x) = 

Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.4932816745772619 km s^-1 Myr^-1, -0.09865633491545238 km s^-1 Myr^-1, 0.19731266983090476 km s^-1 Myr^-1]
potential(pot₂, x) = -1.0202395888647748 km^2 s^-2
acceleration(pot₂, x) = 

Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.09304976990451902 km s^-1 Myr^-1, -0.018609953980903805 km s^-1 Myr^-1, 0.03721990796180761 km s^-1 Myr^-1]


potential(pot + pot₂, x) = Quantity{Float64, 𝐋^2 𝐓^-2, Unitful.FreeUnits{(km^2, s^-2), 𝐋^2 𝐓^-2, nothing}}[-1.0330649124037836 km^2 s^-2, -1.0330649124037836 km^2 s^-2, -1.0330649124037836 km^2 s^-2]
acceleration(pot + pot₂, x) = Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.5863314444817809 km s^-1 Myr^-1, -0.11726628889635618 km s^-1 Myr^-1, 0.23453257779271236 km s^-1 Myr^-1]


In [98]:
@code_warntype  potential(pot,x)

MethodInstance for GalacticDynamics.potential(::Plummer{Int64, Float64}, ::Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(pc,), 𝐋, nothing}}})
  from potential(pot::Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}, x::AbstractArray{<:Union{Quantity{T, 𝐋, U}, Level{L, S, Quantity{T, 𝐋, U}} where {L, S}} where {T, U}}) @ GalacticDynamics ~/.julia/dev/GalacticDynamics/src/geometry/potentials/potentials.jl:23
Arguments
  #self#::Core.Const(GalacticDynamics.potential)
  pot::Plummer{Int64, Float64}
  x::Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(pc,), 𝐋, nothing}}}
Body::Quantity{Float64, 𝐋^2 𝐓^-2, Unitful.FreeUnits{(km^2, s^-2), 𝐋^2 𝐓^-2, nothing}}
1 ─

 %1 = Base.getproperty(GalacticDynamics.𝕦, :t)::Core.Const(0.9777922216807892 Gyr)
│   %2 = (0 * %1)::Core.Const(0.0 Gyr)
│   %3 = GalacticDynamics.potential(pot, x, %2)::Quantity{Float64, 𝐋^2 𝐓^-2, Unitful.FreeUnits{(km^2, s^-2), 𝐋^2 𝐓^-2, nothing}}
└──      return %3



### DifferentialEquations.jl https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

In [99]:
using DifferentialEquations
using Test

function example_orbit()
    m_gal = 2.325e7*u"Msun"
    m =1018.0*m_gal  # Msun
    a = 2.562*u"kpc"     # kpc
    Λ = 200.0*u"kpc"    # kpc
    γ = 2.0
    pot = AllenSantillanHalo(m, a, Λ, γ)
    for i in range(1,20)
        w₀ = 50*rand(6)
        x₀ = w₀[1:3]u"kpc"
        v₀ = w₀[4:6]u"km/s"
        t_range = (0.0,100.0).*𝕦.t
        sol = evolve(pot, x₀, v₀, t_range)
        sol₂ = evolve(pot, x₀, v₀, t_range; options=SolverConfig(solver=Tsit5(), reltol=5.0e-12))
        @show @test sol.x[end] ≈ sol₂.x[end] rtol=5.0e-8
        @show @test sol.v[end] ≈ sol₂.v[end] rtol=5.0e-8
    end
end

example_orbit (generic function with 1 method)

In [28]:
example_orbit()

#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


...

#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


In [29]:
@code_warntype

MethodInstance for example_orbit()
  from example_orbit() @ Main /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:4
Arguments
  #self#::Core.Const(example_orbit)
Locals
  @_2

::Union{Nothing, Tuple{Int64, Int64}}
  pot::AllenSantillanHalo{Float64, Float64, Float64, Float64}
  γ::Float64
  Λ::Quantity{Float64, 𝐋, Unitful.FreeUnits{(kpc,), 𝐋, nothing}}
  a::Quantity{Float64, 𝐋, Unitful.FreeUnits{(kpc,), 𝐋, nothing}}
  m

::Quantity{Float64, 𝐌, Unitful.FreeUnits{(M⊙,), 𝐌, nothing}}
  m_gal::Quantity{Float64, 𝐌, Unitful.FreeUnits{(M⊙,), 𝐌, nothing}}
  value@_9::Union{Test.Error, Test.Fail, Test.Pass}
  value@_10::Union{Test.Error, Test.Fail, Test.Pass}
  i::Int64
  sol₂::Any
  sol::Any
  t_range

::Tuple{Quantity{Float64, 𝐓, Unitful.FreeUnits{(Gyr,), 𝐓, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(Gyr,), 𝐓, nothing}}}
  v₀::Vector{Quantity{Float64, 𝐋 𝐓^-1, Unitful.FreeUnits{(km, s^-1), 𝐋 𝐓^-1, nothing}}}
  x₀::Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(kpc,), 𝐋, nothing}}}
  w₀::Vector{Float64}
  _do@_18::typeof(Test.do_test)
  _e@_19::Any
  _do@_20::typeof(Test.do_test)
  _e@_21::Any
  @_22::Union{Test.Error, Test.Fail, Test.Pass}
  @_23::typeof(Test.do_test)
  @_24::Union{Test.Returned, Test.Threw}
  @_25::Union{Test.Error, Test.Fail, Test.Pass}
  @_26::typeof(Test.do_test)
  @_27::Union{Test.Returned, Test.Threw}
Body::Nothing
1 ──

(m_gal = 2.325e7 * 

...

::Core.Const(SolverConfig
  solver: Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}
  abstol: Float64 5.0e-11
  reltol: Float64 5.0e-12
)
│    %42  = Core.tuple(%41)::Core.Const((SolverConfig
  solver: Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}
  abstol: Float64 5.0e-11
  reltol: Float64 5.0e-12
,))
│    %43  = (%35)(%42)::Core.Const((options = SolverConfig
  solver: Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}
  abstol: Float64 5.0e-11
  reltol: Float64 5.0e-12
,))
│           (sol₂ = Core.kwcall(%43, Main.evolve, pot::Core.Const(AllenSantillanHalo{Float64, Float64, Float64, Float64}
  m: Float64 2.36685e10
  a: Float64 2.562
  Λ: Float64 200.0
  γ: Float64 2.0
), x₀, v₀, t_range::Core.Const((0.0 Gyr, 97.77922216807892 Gyr))))
└───        goto #4 if not false
3 ──        Core.Const(:(Test.get_testset()))


│           Core.Const(:(Test.Broken(:skipped, $(QuoteNode(:(≈(sol.x[end], sol₂.x[end], rtol = 5.0e-8)))))))
│           Core.Const(:(@_22 = Test.record(%46, %47)))
└───        Core.Const(:(goto %84))
4 ┄─        goto #6 if not false
5 ──        Core.Const(:(@_23 = Test.do_broken_test))
└───        Core.Const(:(goto %54))
6 ┄─        (@_23 = Test.do_test)
│    %54  = @_23::Core.Const(Test.do_test)
└───        (_do@_18 = %54)
7 ── %56  = $(Expr(:enter, #10))
8 ── %57  = Main.:≈::Core.Const(isapprox)
│    %58  = (:rtol => 5.0e-8)::Core.Const(:rtol => 5.0e-8)


│    %59  = Test.Expr(:parameters, %58)::Expr
│    %60  = Base.getproperty(sol, :x)::Any
│    %61  = Base.lastindex(%60)::Any
│    %62  = Base.getindex(%60, %61)::Any
│    %63  = Base.getproperty(sol₂, :x)::Any
│    %64  = Base.lastindex(%63)::Any
│    %65  = Base.getindex(%63, %64)::Any
│    %66  = Test.Expr(:call, %57, %59, %62, %65)::Expr
│    %67  = Test.Expr(:call, :≈)::Expr
│           (@_24 = Test.eval_test(%66, %67, $(QuoteNode(:(#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =#))), $(QuoteNode(false))))
└───        $(Expr(:leave, 1))
9 ──        goto #14
10 ┄        $(Expr(:leave, 1))
11 ─        (_e@_19 = $(Expr(:the_exception)))
│    %73  = (_e@_19 isa Test.InterruptException)::Bool
└───        goto #13 if not %73
12 ─        Test.rethrow()
└───        Core.Const(:(goto %77))
13 ┄ %77  = _e@_19::Any
│    %78  = Base.current_exceptions::Core.Const(current_exceptions)
│    %79  = (%78)()::Base.ExceptionStack
│           (@_24 = Test.Threw(%77, %79, $(QuoteNo

::Union{Test.Returned, Test.Threw}
│           (@_22 = (_do@_18)(%82, $(QuoteNode(:(≈(sol.x[end], sol₂.x[end], rtol = 5.0e-8))))))
│           (value@_10 = @_22)
│    %85  = @_22::Union{Test.Error, Test.Fail, Test.Pass}
│    %86  = Base.repr(%85)::String
│           Base.println(

"#= /mnt/md1/mmestre/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = ", %86)
│           value@_10
└───        goto #16 if not false
15 ─        Core.Const(:(Test.get_testset()))
│           Core.Const(:(Test.Broken(:skipped, $(QuoteNode(:(≈(sol.v[end], sol₂.v[end], rtol = 5.0e-8)))))))
│           Core.Const(:(@_25 = Test.record(%90, %91)))
└───        Core.Const(:(goto %128))
16 ┄        goto #18 if not false
17 ─        Core.Const(:(@_26 = Test.do_broken_test))
└───        Core.Const(:(goto %98))
18 ┄        (@_26 = Test.do_test)
│    %98  = @_26::Core.Const(Test.do_test)
└───        (_do@_20 = %98)
19 ─ %100 = $(Expr(:enter, #22))
20 ─ %101 = Main.:≈::Core.Const(isapprox)
│    %102 = (:rtol => 5.0e-8)::Core.Const(:rtol => 5.0e-8)
│    %103 = Test.Expr(:parameters, %102)::Expr
│    %104 = Base.getproperty(sol, :v)::Any
│    %105 = Base.lastindex(%104)::Any
│    %106 = Base.getindex(%104, %105)::Any
│    %107 = Base.getproperty(sol₂, :